<a href="https://colab.research.google.com/github/omarSamy212/Electro_PI/blob/main/Airbnb_Listing_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
df = pd.read_csv('listings.csv')

FileNotFoundError: ignored

In [ ]:
columns_to_keep = [
    'id', 'name', 'latitude', 'longitude', 'property_type', 'room_type',
    'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price',
    'minimum_nights', 'maximum_nights', 'availability_30', 'availability_60',
    'availability_90', 'availability_365', 'number_of_reviews',
    'review_scores_rating', 'instant_bookable', 'reviews_per_month','neighbourhood_cleansed'
]

df = df[columns_to_keep]

In [ ]:
# Set 'id' as the primary key
df.set_index('id', inplace=True)


In [ ]:
# Handle missing values
df['bathrooms'].fillna(0, inplace=True)
df['bedrooms'].fillna(0, inplace=True)


In [ ]:
mean_rating = df['review_scores_rating'].mean()
df['review_scores_rating'].fillna(mean_rating, inplace=True)


In [ ]:
mean_rating_permonth = df['reviews_per_month'].mean()
df['reviews_per_month'].fillna(mean_rating, inplace=True)


In [ ]:
df.columns.unique()

In [ ]:
df['availability_60'].unique()

In [ ]:
# Encode 'instant_bookable' column
df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0})

In [ ]:
df.head()

In [ ]:
df['property_type'].unique()


In [ ]:
df.head()

In [ ]:
# Clean the 'price' column and convert it to float
df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)

In [ ]:
#Feature Selection

selected_features = [
    'price',
    'availability_30',
    'availability_60',
    'availability_90',
    'availability_365',
    'neighbourhood_cleansed',
    'property_type',
    'room_type',
    'accommodates',
    'bedrooms',
    'bathrooms',
    'beds',
    'number_of_reviews',
    'review_scores_rating',
    'instant_bookable'
]

df = df[selected_features]

In [ ]:
df.head()

In [ ]:
 Automated Data Visualization
def visualize_data(df, column):
    if df[column].dtype == 'object':
        plt.figure(figsize=(12, 6))
        sns.countplot(data=df, x=column)
        plt.title(f'Counts of {column}')
        plt.xticks(rotation=45)
        plt.show()
    else:
        plt.figure(figsize=(12, 6))
        sns.histplot(data=df, x=column, bins=30, kde=True)
        plt.title(f'Distribution of {column}')
        plt.xlabel(column)
        plt.ylabel('Frequency')
        plt.show()


In [ ]:
# Visualize selected columns
for column in df.columns:
    visualize_data(df, column)

In [ ]:
# Correlation Heatmap
correlation_matrix = df.corr()
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Statistical Analysis
summary_statistics = df.describe()
correlation_coefficients = df.corr()

In [ ]:
# Test price difference between room types
room_types = df['room_type'].unique()
ttest_results = {}
for room_type in room_types:
    subset = df[df['room_type'] == room_type]
    tstat, pvalue = stats.ttest_ind(subset['price'], df['price'])
    ttest_results[room_type] = {'t-statistic': tstat, 'p-value': pvalue}

In [ ]:
# Test association between 'room_type' and 'property_type'
contingency_table = pd.crosstab(df['room_type'], df['property_type'])
chi2, p, _, _ = stats.chi2_contingency(contingency_table)

In [ ]:
# Insights and Benefits
"""
Insights:
1. The distribution of 'price' is right-skewed, indicating that most listings have lower prices.
2. 'Entire home/apt' room type tends to have higher prices compared to other room types.
3. There's no strong linear correlation between numeric variables like 'price', 'availability', and 'review_scores_rating'.
4. 'Entire home/apt' is the most common room type.
5. The statistical tests suggest that there is a significant price difference between different room types.
6. The chi-square test shows that 'room_type' and 'property_type' are not independent, indicating an association between them.

Benefits:
1. Understanding the distribution of 'price' helps users make pricing decisions.
2. Visualizing 'room_type' vs. 'price' provides insights into pricing strategies.
3. The correlation heatmap helps identify potential relationships between variables.
4. Statistical tests validate hypotheses about price differences and associations between categorical variables.
5. Insights assist hosts and Airbnb users in making informed decisions.
"""

In [ ]:
# Display results
for room_type, results in ttest_results.items():
    print(f"{room_type}: t-statistic = {results['t-statistic']}, p-value = {results['p-value']}")
print(f"Chi-Square Statistic: {chi2}")
print(f"P-value: {p}")